In [1]:
!pip install ezdxf

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import ezdxf
import numpy
import cv2
import argparse

In [3]:
def img_to_dxf(
    input_path,
    output_path="output.dxf",
    downscale=1.0,
    blur=3,
    threshold=127,
    use_canny=False,
    min_points=8,
    approx_epsilon=1.5,
    lower=None,
    upper=None
):
    """
    Convert an input image to DXF using contour tracing.
    """
    def parse_tuple(s):
        s = s.replace("(", "").replace(")", "")
        parts = [p.strip() for p in s.split(",")]
        return tuple(int(p) for p in parts)

    # Load image
    img = cv2.imread(input_path, cv2.IMREAD_COLOR)
    if img is None:
        raise ValueError(f"Could not read image '{input_path}'")

    # Optional color mask
    if lower and upper:
        lower = np.array(parse_tuple(lower), dtype=np.uint8)
        upper = np.array(parse_tuple(upper), dtype=np.uint8)
        mask = cv2.inRange(img, lower, upper)
        proc = mask
    else:
        proc = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Downscale
    if downscale > 1.0:
        new_w = int(proc.shape[1] / downscale)
        new_h = int(proc.shape[0] / downscale)
        proc = cv2.resize(proc, (new_w, new_h), interpolation=cv2.INTER_AREA)

    # Blur
    if blur and blur > 1:
        k = blur if blur % 2 == 1 else blur + 1
        proc = cv2.GaussianBlur(proc, (k, k), 0)

    # Edge detection / thresholding
    if use_canny:
        if len(proc.shape) == 3:
            gray = cv2.cvtColor(proc, cv2.COLOR_BGR2GRAY)
        else:
            gray = proc
        bin_img = cv2.Canny(gray, 50, 150)
    else:
        if len(proc.shape) == 3:
            gray = cv2.cvtColor(proc, cv2.COLOR_BGR2GRAY)
        else:
            gray = proc
        _, bin_img = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY_INV)

    # Contours
    contours_info = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours_info[0] if len(contours_info) == 2 else contours_info[1]

    # Create DXF
    doc = ezdxf.new('R2010')
    msp = doc.modelspace()
    h, w = bin_img.shape[:2]

    total_polylines = 0
    for cnt in contours:
        if len(cnt) < min_points:
            continue
        approx = cv2.approxPolyDP(cnt, approx_epsilon, closed=True)
        pts = []
        for p in approx.reshape(-1, 2):
            x = float(p[0])
            y = float(h - p[1])  # flip Y
            if downscale > 1.0:
                x *= downscale
                y *= downscale
            pts.append((x, y))
        if len(pts) >= 2:
            msp.add_lwpolyline(pts, close=True)
            total_polylines += 1

    doc.saveas(output_path)
    print(f"Saved '{output_path}' with {total_polylines} polylines (image size: {w}x{h})")


In [4]:
img_to_dxf(
    input_path="drawing1.jpg",   # Your drawing image
    output_path="output.dxf",     # Output DXF file
    downscale=2.0,                # Optional: reduce complexity
    blur=3,
    threshold=127,
    use_canny=True,               # Try edge detector
    min_points=10,
    approx_epsilon=2.0
)


Saved 'output.dxf' with 25 polylines (image size: 400x258)
